This Kernal is Cloned one of Mercari Interactive EDA + Topic Modelling by ThyKhueLy.<br />
So, I'd like to give BIG APPALUSE to him, Thanks.
> https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling

# Introduction <br />
This is an initial Explanatory Data Analysis for the Mercari Price Suggestion Challenge with matplotlib. <br />
=> 이 커널은 해당 컴피티션의 가장 기초적인 EDA분석입니다 <br />
bokeh and Plot.ly - a visualization tool that creates beautiful interactive plots and dashboards.<br />
=> boken 과 Plot.ly는 시각화 툴을 좀더 상호작용을 높여주는 툴입니다. <br />
The competition is hosted by Mercari, the biggest Japanese community-powered shopping app with the main objective to predict an accurate price that Mercari should suggest to its sellers, given the item's information. <br />
=> 해당 컴피티션은 일본에서 가장 큰 쇼핑앱이며 우리의 목표는 회사가 판매자들에게 아이템의 정보를 받았을 때 그 상품의 적절한 가격을 제안해 주는 것입니다. <br />

## Update: The abundant amount of food from my family's Thanksgiving dinner has really energized me to continue working on this model. <br />
I decided to dive deeper into the NLP analysis and found an amazing tutorial by Ahmed BESBES. <br />
=> Ahmed BESBES 의 NLP분석을 참고하여 추가작업을 했습니다. <br />
The framework below is based on his source code. <br />
=> 추가된 것들은 그의 코드를 기반으로 작성하였습니다. <br />
It provides guidance on pre-processing documents and machine learning techniques (K-means and LDA) to clustering topics. So that this kernel will be divided into 2 parts: <br />
=> 그의 코드는 문서의 전처리와 clustering topic들에 머신러닝 기술들(K-means 와 LDA)의 가이드가 될 것입니다. 따라서 이 커널은 크게 두가지로 나뉩니다 <br />

### Explanatory Data Analysis <br />

### Text Processing <br />
#### 2.1. Tokenizing and tf-idf algorithm <br />
#### 2.2. K-means Clustering <br />
#### 2.3. Latent Dirichlet Allocation (LDA) / Topic Modelling <br />

In [ ]:
import nltk
import string
import re
import numpy as np
import pandas as pd
import pickle
#import lda

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")

from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words

from collections import Counter
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
#from bokeh.transform import factor_cmap

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

PATH = "../input/"

In [ ]:
len(stopwords.words('english'))

In [ ]:
len(stop_words.ENGLISH_STOP_WORDS)

In [ ]:
train = pd.read_csv(f"{PATH}train.tsv",sep='\t')
test = pd.read_csv(f"{PATH}test.tsv",sep='\t')

우리가 사용하게 될 데이터셋의 생김새는 아래와 같다.

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
frac = 0.1
train = train.sample(frac=frac,random_state=200)
test = test.sample(frac=frac,random_state=200)

In [ ]:
print(train.shape)
print(test.shape)

또한 훈련데이터셋에서 데이터프레임 각각의 컬럼들의 타입은 아래와 같다.

### The files consist of a list of product listings. These files are tab-delimited. <br />

* **train_id or test_id - the id of the listing <br />**
=> 목록의 고유한 id <br />
* **name - the title of the listing. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm] <br />**
=> 목록에 올라온 제목. 가격처럼 생긴 텍스트는 데이터에서 지우려고 하고 있습니다. 이렇게 지워진 가격들은 [rm]이라고 표시되어 있습니다.<br />

* **item_condition_id - the condition of the items provided by the seller <br />**
=> 판매자에게 제공받은 물품의 상태를 나타냄<br />
* **category_name - category of the listing <br />**
=> 목록의 카테고리<br />
* **brand_name <br />**
=> 브랜드 이름 <br />
* **price - the price that the item was sold for. This is the target variable that you will predict. The unit is USD. This column doesn't exist in test.tsv since that is what you will predict. <br />**
=> 아이템이 팔린 가격. 해당 컬럼이 우리가 예측하고자하는 타겟 컬럼이다. 단위는 미국 달러이며. 해당 컬럼은 test.tsv 에는 따로 기재되어 있지 않다,
* **shipping - 1 if shipping fee is paid by seller and 0 by buyer <br />**
=> 1이면 판매자가 운임을 냈으며 0이면 구매자가 처리한다<br />
* **item_description - the full description of the item. Note that we have cleaned the data to remove text that look like prices (e.g. $20) to avoid leakage. These removed prices are represented as [rm] <br />**
=> 물품의 상세내용. 이 데이터 또한 name에서 처럼 가격을 지우려고 했으며 지워진 물품에 대해서는 [rm] 표기되어 있습니다. <br />

Please note that in stage 1, all the test data will be calculated on the public leaderboard. In stage 2, we will swap the test.tsv file to the complete test dataset that includes the private leaderboard data.

In [ ]:
train.dtypes

In [ ]:
train.head()

## Target Variable: Price <br />
The next standard check is with our response or target variables, which in this case is the price we are suggesting to the Mercari's marketplace sellers. <br />
=> 다음으로 확인해봐야 할 것은 Price컬럼입니다. <br />
The median price of all the items in the training is about \$267 but given the existence of some extreme values of over \$100 and the maximum at \$2,009, the distribution of the variables is heavily skewed to the left. <br />
=> 가격의 중앙값은 267달러이지만, 어떤 가격은 100달러를 웃돌거나 최대로는 2009달러 짜리 물품도 있습니다. 해당 가격의 분포는 굉장히 좌측으로 치우쳐 있습니다. <br />
So let's make log-transformation on the price (we added +1 to the value before the transformation to avoid zero and negative values). <br />
=> 따라서 가격에 대해서 로그화를 진행해 봅시다. <br />

In [ ]:
train['price'].describe()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=[10,5])
ax = plt.subplot(1,2,1)
sns.distplot(train['price'],hist=False,ax=ax,bins=50)
ax.set(ylabel='Density',title='Distribution of price')
ax = plt.subplot(1,2,2)
sns.distplot(np.log1p(train['price']),hist=False,ax=ax,bins=50)
ax.set(xlabel='log1p(price)',ylabel='Density',title='Distribution of log1p(price)')

## Shipping <br />
The shipping cost burden is decently splitted between sellers and buyers with more than half of the items' shipping fees are paid by the sellers (55%). <br />
부담하는 배송비는 판매자와 구매자가 거의 반반으로 나누어져 있습니다. <br />

In addition, the average price paid by users who have to pay for shipping fees is lower than those that don't require additional shipping cost. <br />
=> 게다가 배송비로 사용자가 지불해야하는 돈의 평균은 추가적인 배송비를 필요로하지 않는 것보다 낮았습니다. <br />
This matches with our perception that the sellers need a lower price to compensate for the additional shipping. <br />
=> 이에 따르면 판매자들은 낮은 가격을 배송비를 추가함으로써 무마하려는 것 같습니다. <br />

In [ ]:
train['shipping'].value_counts()

In [ ]:
shipping_buyer_price = train.loc[train['shipping'] == 0,'price']
shipping_seller_price = train.loc[train['shipping'] == 1,'price']

fig = plt.figure(figsize=[10,5])
sns.distplot(np.log1p(shipping_buyer_price),bins=50,kde=False,color='red',label='buyer')
sns.distplot(np.log1p(shipping_seller_price),bins=50,kde=False,color='blue',label='seller')
plt.xlabel('log1p(price)')
plt.legend()

## Item Category <br />
There are about 1,287 unique categories but among each of them, we will always see a main/general category firstly, followed by two more particular subcategories (e.g. Beauty/Makeup/Face or Lips). <br />
=> 1287개의 서로다른 카테고리들이 존재한다. 우리는 가장 주된 카테고리를 먼저 볼 수 있으며, 다음으로 두 개의 다른 카테고리를 더 볼 수 있다. <br />
In adidition, there are about 6,327 items that do not have a category labels. <br />
=> 그러나 6327개의 물품들이 카테고리가 존재하지 않음을 알 수 있다. <br />

Let's split the categories into three different columns. <br />
=> 일단 카테고리들을 3개의 컬럼으로 나누어 보자 <br />
We will see later that this information is actually quite important from the seller's point of view and how we handle the missing information in the brand_name column will impact the model's prediction. <br />
=> 나누어진 컬럼들은 추후에 판매자의 입장에서 굉장히 중요하며, 브랜드 이름 컬럼에서 손실값을 어떻게 다루는지가 모델의 예측을 다르게 할 것입니다.

In [ ]:
train['category_name'].nunique()

In [ ]:
train['category_name'].isnull().sum()

In [ ]:
category_null = train.loc[train['category_name'].isnull()]
category_notnull = train.loc[train['category_name'].notnull()]

# print(category_notnull.shape)
# category_notnull.info()

sep_categories = category_notnull['category_name'].apply(lambda x:x.split('/'))

col_names = ['general_cat','subcat_1','subcat_2']

cats = []

for cats_ in sep_categories:
    cat_dict = {col_names[0]:cats_[0],col_names[1]:cats_[1],col_names[2]:cats_[2]}
    cats.append(cat_dict)
    
df_cats = pd.DataFrame(cats)

category_notnull = category_notnull.reset_index(drop=True)
category_notnull = pd.concat([category_notnull,df_cats],axis=1)

for col in col_names:
    category_null[col] = 'No Label'

# assert category_notnull.index.max() == category_notnull.train_id.max()

train = pd.concat([category_notnull,category_null],axis=0,sort=False).sort_values(by='train_id').reset_index(drop=True)
train.info()

In [ ]:
# # reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
# def split_cat(text):
#     try: return text.split("/")
#     except: return ("No Label", "No Label", "No Label")

In [ ]:
# train['general_cat'], train['subcat_1'], train['subcat_2'] = \
# zip(*train['category_name'].apply(lambda x: split_cat(x)))
# train.head()

In [ ]:
print("There are %d unique general-categories." % train['general_cat'].nunique())

In [ ]:
print("There are %d unique first sub-categories." % train['subcat_1'].nunique())

In [ ]:
print("There are %d unique second sub-categories." % train['subcat_2'].nunique())

In [ ]:
train['general_cat']

In [ ]:
plt.figure(figsize=[8,6])
train['general_cat'].value_counts().plot(kind='bar',linewidth=2)
plt.title('Number of Items by Main Category')
plt.ylabel('Count')
plt.xlabel('Categories')
plt.xticks(rotation=60)

In [ ]:
train['subcat_1'].value_counts()[:15].plot(kind='bar',linewidth=2, figsize=[12,10])
plt.title('Number of Items by Subset_1')
plt.ylabel('Count')
plt.xlabel('Categories')
plt.xticks(rotation=60)

In [ ]:
fig = plt.figure(figsize=[12,10])
sns.boxplot(x=train['general_cat'],y=np.log1p(train['price']))
plt.xticks(rotation=60)

In [ ]:
def wordCount(text):
    # convert to lower case and strip regex
    try:
         # convert to lower case and strip regex
        text = text.lower()
        #regular expression pattern 생성
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        #패턴을 이용하여 들어오는 텍스트에서 해당 패턴의 기호들을 제거하는 것
        txt = regex.sub(" ", text)
        # tokenize
        # words = nltk.word_tokenize(clean_txt)
        # remove words in stop words
        words = [w for w in txt.split(" ") \
                 if not w in stop_words.ENGLISH_STOP_WORDS and len(w)>3]
        return len(words)
    except: 
        return 0

In [ ]:
train['item_description'][3]

In [ ]:
regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')

In [ ]:
regex.sub(" ",train['item_description'][3])

In [ ]:
list(stop_words.ENGLISH_STOP_WORDS)[:5]

In [ ]:
# add a column of word counts to both the training and test set
train['desc_len'] = train['item_description'].apply(lambda x: wordCount(x))
test['desc_len'] = test['item_description'].apply(lambda x: wordCount(x))

In [ ]:
train.head()

In [ ]:
df = train.groupby('desc_len')['price'].mean().reset_index()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=[20,8])
sns.pointplot(x=df.columns[0],y=df.columns[1],data=df)
plt.xticks(rotation=60)

We also need to check if there are any missing values in the item description (4 observations don't have a description) and l remove those observations from our training set. <br />
=> 아이템 설명 란에 다른 손실값이 있는지 찾아봅시다. 4개의 손실값을 찾을 수 있었으며 우리의 훈련셋에서 제외합시다.

In [ ]:
train['item_description'].isnull().sum()

In [ ]:
train = train.loc[train['item_description'].notnull()]

In [ ]:
stop = set(stopwords.words('english'))
def tokenize(text):
    """
    sent_tokenize(): segment text into sentences
    word_tokenize(): break sentences into words
    """
    try: 
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        text = regex.sub(" ", text) # remove punctuation
        
        tokens_ = [word_tokenize(s) for s in sent_tokenize(text)]

        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent

        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        filtered_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
        filtered_tokens = [w.lower() for w in filtered_tokens if len(w)>=3]
        
        return filtered_tokens
            
    except TypeError as e: print(text,e)

In [ ]:
cat_desc = dict()
for cat in train.general_cat.unique(): 
    text = " ".join(train.loc[train['general_cat']==cat, 'item_description'].values)
    cat_desc[cat] = tokenize(text)

In [ ]:
# flat list of all words combined
flat_lst = [item for sublist in list(cat_desc.values()) for item in sublist]
allWordsCount = Counter(flat_lst)
all_top10 = allWordsCount.most_common(20)
x = [w[0] for w in all_top10]
y = [w[1] for w in all_top10]

In [ ]:
len(flat_lst)

In [ ]:
len(allWordsCount)

In [ ]:
all_top10

If we look at the most common words by category, we could also see that, size, free and shipping is very commonly used by the sellers, probably with the intention to attract customers, which is contradictory to what we have shown previously that there is little correlation between the two variables price and shipping (or shipping fees do not account for a differentiation in prices). <br />
=> 카테고리에 따라 가장 자주나온 단어를 본다면, 우리는 size,free 그리고 shipping 이라는 단어들이 판매자들이 소비자들의 관심을 얻기위해 주로 사용합니다. 이러한 가격과 배송의 연관관계는 앞에서 살펴보았습니다. (또는 배송비는 가격을 결정하는데 작용하지 않을 수도 있습니다.) <br />
Brand names also played quite an important role - it's one of the most popular in all four categories. <br />
=> 브랜드이름은 꽤나 중요한 역할을 합니다 - 네개의 카테고리들 중에 가장 빈번한 것들중 하나입니다.

Text Processing - Item Description <br />
=> 물품 상세 컬럼을 위한 문자 전처리 <br />
The following section is based on the tutorial at https://ahmedbesbes.com/how-to-mine-newsfeed-data-and-extract-interactive-insights-in-python.html

## **Pre-processing:  tokenization**

Most of the time, the first steps of an NLP project is to **"tokenize"** your documents, which main purpose is to normalize our texts. The three fundamental stages will usually include: </br>
=> 자연어 처리의 첫 번째 해야할 일은 문서를 tokenize하는 것입니다. (우리의 문자들을 정규화하는 것이죠). 이는 대부분 세 개의 절차로 나누어져 있습니다.
* break the descriptions into sentences and then break the sentences into tokens <br />
=> 설명을 문장별로 나누고 그 문장을 토큰화합니다 <br />
* remove punctuation and stop words <br />
=> 각종 따옴표등을 없애고 단어들로 만듭니다 <br />
* lowercase the tokens
=> 토큰들을 소문자화 합니다 <br />
* herein, I will also only consider words that have length equal to or greater than 3 characters <br />
=>여기에서는 우리는 3글자보다 크거나 같은 길이를 가진 단어들만 고려하면 됩니다.

In [ ]:
# stop = set(stopwords.words('english'))
# def tokenize(text):
#     """
#     sent_tokenize(): segment text into sentences
#     word_tokenize(): break sentences into words
#     """
#     try: 
#         regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
#         text = regex.sub(" ", text) # remove punctuation
        
# #         print(sent_tokenize(text))
        
#         tokens_ = [word_tokenize(s) for s in sent_tokenize(text)]
# #         print(tokens_)
#         tokens = []
#         for token_by_sent in tokens_:
#             tokens += token_by_sent
# #         print(tokens)
#         tokens = list(filter(lambda t: t.lower() not in stop, tokens))
# #         print(tokens)
#         filtered_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
# #         print(filtered_tokens)
#         filtered_tokens = [w.lower() for w in filtered_tokens if len(w)>=3]
# #         print(filtered_tokens)
        
#         return filtered_tokens
            
#     except TypeError as e: print(text,e)

In [ ]:
train['tokens'] = train['item_description'].apply(tokenize)
test['tokens'] = train['item_description'].apply(tokenize)

In [ ]:
for description, tokens in zip(train['item_description'].head(),
                              train['tokens'].head()):
    print('description:', description)
    print('tokens:', tokens)
    print()

In [ ]:
#어짜피 토큰 만들어놨는데 다시 조인해서 토크나이즈 하는 것은 낭비라고 생각
cat_desc = dict()
for cat in train.general_cat.unique():
    cat_list = []
    for token in train.loc[train['general_cat']==cat,'tokens']:
        for one in token:
            cat_list.append(one)
    cat_desc[cat] = cat_list

In [ ]:
women100 = Counter(cat_desc['Women']).most_common(100)
beauty100 = Counter(cat_desc['Beauty']).most_common(100)
kids100 = Counter(cat_desc['Kids']).most_common(100)
electronics100 = Counter(cat_desc['Electronics']).most_common(100)

In [ ]:
women100

In [ ]:
def generate_wordcloud(tup):
    wordcloud = WordCloud(background_color='white',max_words=50,max_font_size=40,random_state=42).generate(str(tup))
    return wordcloud

In [ ]:
fig,ax = plt.subplots(2,2,figsize=[30,15])

ax = plt.subplot(2,2,1)
ax.imshow(generate_wordcloud(women100),interpolation="bilinear")
ax.axis('off')

ax = plt.subplot(2,2,2)
ax.imshow(generate_wordcloud(beauty100))
ax.axis('off')

ax = plt.subplot(2,2,3)
ax.imshow(generate_wordcloud(kids100))
ax.axis('off')

ax = plt.subplot(2,2,4)
ax.imshow(generate_wordcloud(electronics100))
ax.axis('off')

In [ ]:
train.head()

## **Pre-processing:  tf-idf**

tf-idf is the acronym for **Term Frequency–inverse Document Frequency**. It quantifies the importance of a particular word in relative to the vocabulary of a collection of documents or corpus. The metric depends on two factors: <br />
=> tf-idf는 "Term Frequency-inverse Document Frequency"의 약어입니다. 이는 문서나 말뭉치의 집합의 단어와 관련된 특정언어의 중요도를 수치화합니다 <br />
- **Term Frequency**: the occurences of a word in a given document (i.e. bag of words)
=> 주어진 문서에서 단어의 등장 정도<br />
- **Inverse Document Frequency**: the reciprocal number of times a word occurs in a corpus of documents
=>단어의 뭉치에서 단어가 등장한 상호간의 횟수<br />

Think about of it this way: If the word is used extensively in all documents, its existence within a specific document will not be able to provide us much specific information about the document itself. <br />
=> 만약 단어가 모든 문서에서 많이 사용된다면, 어느 특정한 문서 내의 해당 단어의 존재는 그 문서에 대해 특정한 정보라고 제시할 수 없습니다.
So the second term could be seen as a penalty term that penalizes common words such as "a", "the", "and", etc. tf-idf can therefore, be seen as a weighting scheme for words relevancy in a specific document. <br />
=> 그래서 두번째로 제약을 거는데, 많이 등장하는 언어 a,the,and 등에. 그러므로 tf-idf는 어느 특정한 문서에 단어들의 연관성에 가중치를 두는것과 유사합니다.


## ** tf-idf: 단어빈도-역문서빈도** [얼마나 의미 있는 특성인지를 계산해서 스케일을 조정하는 방식]
**말뭉치의 다른 문서보다 특정 문서에 자주 나타나는 단어에 높은 가중치를 주는 방법.** <br />
ex)) 한 단어가 특정 문서에 자주 나타나고 다른 여러 문서에는 그렇지 않다면, 그 문서의 내용을 잘 설명하는 단어라고 볼 수있음.

### Term Frequency: 주어진 다큐먼트(하나의 문자열)에 단어의 등장횟수(CountVectorizer)
### Inverse Document Frequency: 다큐먼트의 하나의 말뭉치에서 임의의 단어가 상호간에 등장하는 횟수
=> CountVectorizer를 혼용하는 것 같음
<br />

e.g) 임의의 단어가 모든 다큐먼트들에서 빈번하기 나온다면, 어느 특정한 다큐먼트에서 그 단어의 존재는 그 다큐먼트에서 해당 단어가 특정한 정보를 제공하지 못한다고 할수있다. <br />
=> 만약 영화에 대한 평가 텍스트라면 서로 다른 영화에서 나올 수 있는 평가인 "그리고" 라는 토큰은 특정 영화가 어느 영화인지 가늠할 수 없지만, "스릴러"라는 토큰은 멜로, 음악등의 영화에서는 등장하지 않는 어느 특정영화에만 나오는 토큰이기 때문에 어느 영화인지 가늠이 가능하다.

따라서 위와 같이 **여러 다큐먼트에서 나오는 토큰들의 경우에는 규제를 가함으로써 가중치를 낮춘다.** 이 때문에 가중치가 높은 단어는 어느 특정 다큐먼트와 관련이 있을 가능성이 커지는 것이다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10,max_features=180000,tokenizer=tokenize,ngram_range=(1,2))

In [ ]:
all_desc = np.append(train['item_description'].values,test['item_description'].values)
vz = vectorizer.fit_transform(list(all_desc))

vz is a tfidf matrix where: <br />
=> tfidf 행렬은 아래와 같습니다 <br />
* the number of rows is the total number of descriptions <br />
=> 행은 전체 설명의 개수 <br />
* the number of columns is the total number of unique tokens across the descriptions <br />
=> 열은 설명들에 있어서 유일한 토큰의 개수를 의미한다

In [ ]:
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(
                    dict(tfidf), orient='index')
tfidf.columns = ['tfidf']

Below is the 10 tokens with the lowest tfidf score, which is unsurprisingly, very generic words that we could not use to distinguish one description from another. <br />
=> tfidf 점수가 낮은 10개의 토큰들은 다른 물품들을 구별할만한 설명이 아닌 단어들이었습니다.

In [ ]:
tfidf.sort_values(by='tfidf',ascending=True).head(10)

Below is the 10 tokens with the highest tfidf score, which includes words that are a lot specific that by looking at them, we could guess the categories that they belong to:  <br />
=> tfidf 점수가 높은 10개의 토큰은 물품들의 설명하는데 있어서 특징적인 것들이 많습니다.

In [ ]:
tfidf.sort_values(by='tfidf',ascending=False).head(10)

Given the high dimension of our tfidf matrix, we need to reduce their dimension using the Singular Value Decomposition (SVD) technique. <br />
=> 주어진 고차원의 tfidf 행렬을 가지고, 우리는 SVD를 활용하여 그 차원을 축소할 수 있습니다. <br />
And to visualize our vocabulary, we could next use t-SNE to reduce the dimension from 50 to 2. t-SNE is more suitable for dimensionality reduction to 2 or 3. <br /> 
=> 그리고 우리의 단어들을 시각화하고, t-SNE를 50차원에서 2차원으로 줄이기 위해 사용할 것입니다. t-SNE는 2차원 또는 3차원으로 줄이는데 가장 적절한 방법입니다.

### **t-Distributed Stochastic Neighbor Embedding (t-SNE)**

t-SNE is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. <br />
=>t-SNE는 특히 고차원의 데이터셋을 시각화하는데 잘 맞는 차원 축소 기술입니다 <br />
The goal is to take a set of points in a high-dimensional space and find a representation of those points in a lower-dimensional space, typically the 2D plane. <br />
=> 목표는 고차원공간에 있는 점들의 집합을 가지고 특히 2차원 공간에 있는 것과 같은 저차원의 점들의 표현법을 찾아내는 것입니다. <br />
It is based on probability distributions with random walk on neighborhood graphs to find the structure within the data. <br />
=> 이는 neighborhood 그래프에서 random walk를 가지고 데이터의 구조를 찾기 위한 확률 분포를 기초로 합니다. <br />
But since t-SNE complexity is significantly high, usually we'd use other high-dimension reduction techniques before applying t-SNE. <br />
=>그러나 t-SNE의 복잡도가 상당히 높다면, t-SNE를 적용하기전에 다른 고차원 축소 기술들을 사용할 수 있습니다. <br />

First, let's take a sample from the both training and testing item's description since t-SNE can take a very long time to execute.<br />
=> 일단은 훈련과 테스트셋의 아이템 설명에서 샘플을 가져와봅시다. 왜냐하면 t-SNE는 실행에 오래걸리기 때문입니다.<br />
We can then reduce the dimension of each vector from to n_components (50) using SVD. <br />
=> 이후에 SVD를 이용하여 차원을 줄여볼 것입니다.


## 차원 축소를 하는 이유?
### **1. 시각화**
#### 2. 데이터 압축
#### 3. 추가적 처리
<br />

차원축소 방법: **주성분분석(PCA), 비음수 행렬 분해(NMF), t-SNE**

In [ ]:
trn = train.copy()
tst = test.copy()
trn['is_train'] = 1
tst['is_train'] = 0

#t-SNE가 시간이 많이 걸리는 작업이기 때문에 임의의 샘플사이즈로 나누고 이를 차원분할해보자
sample_sz = 15000

combined_df = pd.concat([trn,tst])
combined_sample = combined_df.sample(n=sample_sz)
vz_sample = vectorizer.fit_transform(list(combined_sample['item_description']))

In [ ]:
vz_sample.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_comp = 30
svd = TruncatedSVD(n_components=n_comp, random_state=42)
svd_tfidf = svd.fit_transform(vz_sample)

In [ ]:
svd_tfidf.shape

Now we can reduce the dimension from 50 to 2 using t-SNE!

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=42, n_iter=500)

In [ ]:
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)

In [ ]:
tsne_tfidf

It's now possible to visualize our data points. Note that the deviation as well as the size of the clusters imply little information  in t-SNE.

In [ ]:
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600,
                       title="tf-idf clustering of the item description",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
combined_sample = combined_sample.reset_index(drop=True)

In [ ]:
tfidf_df = pd.DataFrame(tsne_tfidf, columns=['x', 'y'])
tfidf_df['description'] = combined_sample['item_description']
tfidf_df['tokens'] = combined_sample['tokens']
tfidf_df['category'] = combined_sample['general_cat']

In [ ]:
tfidf_df.head()

In [ ]:
plot_tfidf.scatter(x='x', y='y', source=tfidf_df, alpha=0.7)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "tokens": "@tokens", "category":"@category"}
show(plot_tfidf)

## **K-Means Clustering** [데이터 셋을 클러스터라는 그룹으로 나누는 작업]

K-means clustering obejctive is to minimize the average squared Euclidean distance of the document / description from their cluster centroids. <br />
=> 위의 목표는 문서나 설명의 cluster centroid의 Euclidean 거리 제곱의 평균을 줄이는 것


In [ ]:
from sklearn.cluster import MiniBatchKMeans

num_clusters = 30 # need to be selected wisely
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters,
                               init='k-means++',
                               n_init=1,
                               init_size=1000, batch_size=1000, verbose=0, max_iter=1000)

In [ ]:
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)

In [ ]:
# sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()

# for i in range(num_clusters):
#     print("Cluster %d:" % i)
#     aux = ''
#     for j in sorted_centroids[i, :10]:
#         aux += terms[j] + ' | '
#     print(aux)
#     print() 

In [ ]:
# repeat the same steps for the sample
kmeans = kmeans_model.fit(vz_sample)
kmeans_clusters = kmeans.predict(vz_sample)
kmeans_distances = kmeans.transform(vz_sample)
# reduce dimension to 2 using tsne
tsne_kmeans = tsne_model.fit_transform(kmeans_distances)

In [ ]:
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#194196", "#d27c88", "#36422b", "#b68f79"])

In [ ]:
#combined_sample.reset_index(drop=True, inplace=True)
kmeans_df = pd.DataFrame(tsne_kmeans, columns=['x', 'y'])
kmeans_df['cluster'] = kmeans_clusters
kmeans_df['description'] = combined_sample['item_description']
kmeans_df['category'] = combined_sample['general_cat']
#kmeans_df['cluster']=kmeans_df.cluster.astype(str).astype('category')

In [ ]:
plot_kmeans = bp.figure(plot_width=700, plot_height=600,
                        title="KMeans clustering of the description",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
source = ColumnDataSource(data=dict(x=kmeans_df['x'], y=kmeans_df['y'],
                                    color=colormap[kmeans_clusters],
                                    description=kmeans_df['description'],
                                    category=kmeans_df['category'],
                                    cluster=kmeans_df['cluster']))

plot_kmeans.scatter(x='x', y='y', color='color', source=source)
hover = plot_kmeans.select(dict(type=HoverTool))
hover.tooltips={"description": "@description", "category": "@category", "cluster":"@cluster" }
show(plot_kmeans)

## **Latent Dirichlet Allocation**

Latent Dirichlet Allocation (LDA) is an algorithms used to discover the topics that are present in a corpus. <br />
=> LDA는 말뭉치내에 존재하는 토픽들을 찾는데 이용되는 알고리즘입니다.

>  LDA starts from a fixed number of topics. Each topic is represented as a distribution over words, and each document is then represented as a distribution over topics. Although the tokens themselves are meaningless, the probability distributions over words provided by the topics provide a sense of the different ideas contained in the documents. <br />
LDA는 고정된 토픽의 수를 결정하면서 시작하며, 각각의 토픽은 모든 단어들에 걸쳐서 나뉘고, 각각의 다큐먼트는 모든 토픽들에 걸쳐서 나뉩니다. 비록 토큰들이 의미가 없더라도, 토픽별로 제공되는 모든 단어들에 걸친 확률분포는 다른 종류의 아이디어들을 제공합니다.
> 
> Reference: https://medium.com/intuitionmachine/the-two-paths-from-natural-language-processing-to-artificial-intelligence-d5384ddbfc18

Its input is a **bag of words**, i.e. each document represented as a row, with each columns containing the count of words in the corpus. We are going to use a powerful tool called pyLDAvis that gives us an interactive visualization for LDA. 

In [ ]:
cvectorizer = CountVectorizer(min_df=4,
                              max_features=180000,
                              tokenizer=tokenize,
                              ngram_range=(1,2))

In [ ]:
cvz = cvectorizer.fit_transform(combined_sample['item_description'])

In [ ]:
cvz

In [ ]:
lda_model = LatentDirichletAllocation(n_components=100,
                                      learning_method='online',
                                      max_iter=20,
                                      random_state=42)

In [ ]:
X_topics = lda_model.fit_transform(cvz)

In [ ]:
lda_model.components_.shape

In [ ]:
sorting = np.argsort(lda_model.components_,axis=1)[::-1]
feature_names = np.array(cvectorizer.get_feature_names())

In [ ]:
fig, ax = plt.subplots(1,2,figsize=[10,12])
topic_names = ["{:>2} ".format(i)+" ".join(words) for i,words in enumerate(feature_names[sorting[:,:2]])]

for col in [0,1]:
    start = col * 50
    end = (col+1) * 50
    ax[col].barh(np.arange(50),np.sum(X_topics,axis=0)[start:end])
    ax[col].set_yticks(np.arange(50))
    ax[col].set_yticklabels(topic_names[start:end],ha='left',va='top')
    ax[col].invert_yaxis()
    ax[col].set_xlim(0,1000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad=130)
plt.tight_layout()